# <font color='black'>Регрессионный анализ: продолжение, 2024 </font>
## <font color='black'> Меры качества модели, сравнение альтернативных спецификаций </font>
В рамках данного практического занятия мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Возьмем такой массив, так как мы пока не знакомились с моделями для анализа панельных данных.

Стоит отметить, что авторы изучают взаимосвязь уровня коррупции (является откликом в регрессионной модели) и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Мы протестируем данную гипотезу на практическом занятии 2, когда познакомимся с регрессионными моделями с переменными взаимодействия. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Приведен к непрерывной шкале от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения: World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from statsmodels.stats.anova import anova_lm

In [ ]:
lab1 = pd.read_stata('lab1.dta')
lab1 = lab1.dropna()

Оценим две спецификации модели, вложенные друг в друга - они будут различаться на один параметр (коэффициент при переменной взаимодействия логарифма ВВП на душу населения и бывшей британской колонии). Проследим, как изменяются меры коэффициента детерминации и скорректированного коэффициента детерминации с добавлением предиктора.

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit(cov_type = "HC3")
print(m1.summary())

In [ ]:
m2 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc*britcol", data = lab1).fit(cov_type = "HC3")
print(m2.summary())

In [ ]:
print("Model1 R-squared:", m1.rsquared.round(3), "Model1 R-squared adjusted:", m1.rsquared_adj.round(3))
print("Model2 R-squared:", m2.rsquared.round(3), "Model2 R-squared adjusted:", m2.rsquared_adj.round(3))

В то время как $R^2$ увеличивается с добавлением новых предикторов или по крайней мере остается таким же, $R^2_{adj}$ может и уменьшиться (в случае добавленных незначимых предикторов). Для лучшего понимания выведем отдельно таблицу разложения вариации для m2.

In [ ]:
print(anova_lm(m2))

Мы уже обсуждали, что оцененная модель может хорошо объяснять вариацию отклика на обучающей выборке, но при этом плохо работать на данных, не участвовавших в обучении - то есть, тестовой выборке. Для того, чтобы определить, не сталкиваемся ли мы со случаем переобучения, мы разделим массив данных на обучающую и тестовую. Обучим модель m1 на 80% выборки. На тестовой выборке проследим меры качества модели.

In [ ]:
train, test = train_test_split(lab1, test_size = 0.2, random_state = 1)

In [ ]:
X_train = train[["dem", "fp", "stab", "loggdppc", "britcol"]]
y_train = train["cpi"]

X_test = test[["dem", "fp", "stab", "loggdppc", "britcol"]]
y_test = test["cpi"]

In [ ]:
m1.train = LinearRegression().fit(X_train, y_train)
y_pred_train = m1.train.predict(X_train)
R2_train = r2_score(y_train, y_pred_train)

y_pred_test = m1.train.predict(X_test)
R2_test = r2_score(y_test, y_pred_test)

print("Training Sample R-squared:", R2_train)
print("Test Sample R-squared:", R2_test)

Для того, чтобы результаты были более обоснованы и не основывались лишь на одном разбиении массива на тестовую и обучающую выборки, используем k-блочную кросс-валидацию (поделим данные на 5 фолдов, проведем 5 итераций и усредним полученные результаты: на каждой итерации алгоритма модель обучается на 4 фолдах, тестируется на оставшейся 5-ой части выборки).  

In [ ]:
X = lab1[["dem", "fp", "stab", "loggdppc", "britcol"]]
y = lab1["cpi"]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
R2 = []

In [ ]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    m1.cv = LinearRegression()
    m1.cv.fit(X_train, y_train)

    y_pred = m1.cv.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    R2.append(r2)

    print("Fold", fold+1, "R2:", r2)

average_R2 = sum(R2) / len(R2)
print("Average R2:", average_R2)

Сравним альтернативные спецификации моделей m3 и m4 при помощи информационных критериев AIC и BIC.

In [ ]:
m3 = smf.ols(formula = "cpi ~ dem + stab + loggdppc + britcol", data = lab1).fit(cov_type = "HC3")
m4 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc", data = lab1).fit(cov_type = "HC3")

In [ ]:
aic_m3 = m3.aic
aic_m4 = m4.aic

bic_m3 = m3.bic
bic_m4 = m4.bic

print("Model3 AIC:", aic_m3.round(3), "Model4 AIC:", aic_m4.round(3))
print("Model3 BIC:", bic_m3.round(3), "Model4 BIC:", bic_m4.round(3))

In [ ]:
p = len(m3.params)
LL = m3.llf
aicm3 = 2*p - 2*LL
aicm3

In [ ]:
bicm3 = np.log(len(lab1))*p - 2*LL
bicm3

Для вложенных моделей мы можем использовать F-test:

In [ ]:
anovaResults = anova_lm(m1, m2)
print(anovaResults)